## Трансдьюсер

In [ ]:
!curl -sS https://apertium.projectjj.com/apt/install-release.sh | sudo bash
!apt install apertium-all-dev lexd

In [2]:
!lexd eve.lexd > eve.generator.att
!lt-comp rl eve.generator.att eve.analyser.bin
! lexd eve.lexd | hfst-txt2fst -o eve.generator.hfst
! hfst-invert eve.generator.hfst -o eve.analyzer.hfst

main@standard 61455 274938


In [3]:
! hfst-twolc eve.twol -o eve.twol.hfst
! lexd eve.lexd | hfst-txt2fst -o eve.lexd.hfst
#! hfst-fst2strings eve.lexd.hfst
! hfst-compose-intersect eve.lexd.hfst eve.twol.hfst -o eve.generator.hfst
#! hfst-fst2strings eve.generator.hfst
! hfst-invert eve.generator.hfst -o eve.analyzer.hfst

Reading input from eve.twol.
Writing output to eve.twol.hfst.
Reading alphabet.
Reading sets.
Reading rules and compiling their contexts and centers.
Compiling rules.
Storing rules.


Пример использования анализатора

In [46]:
word = "d'utkin"
def get_analysis(word):
    good_word = word.replace("'", "\\'")
    answer = ! echo {good_word} | hfst-lookup eve.analyzer.hfst
    if answer[2].split()[2].endswith('+?'):
        analysis = "IDK"
    else:
        analysis = answer[2].split()[2]
        POS = ['<n>', '<v>', '<adj>', '<adv>', '<phd>', '<ptc>',
               '<cvb>', '<num>', '<pron>', '<refl>', '<det>',
               '<func>', '<qpron>', '<intj>', '<refl>']
        for onepos in POS:
            analysis = analysis.replace(onepos, '')
        analysis = analysis.replace('><', '-')
        analysis = analysis.replace('>', '').replace('<', '')
    return analysis.upper()

get_analysis(word)

"D'U-DIR-POSS-P3-SG"

In [5]:
!hfst-invert eve.lexd.hfst -o eve.inv_seg.hfst
!hfst-compose-intersect -1 eve.lexd.hfst -2 eve.twol.hfst | hfst-minimise -o eve-seg.hfst
!hfst-compose -1 eve.inv_seg.hfst -2 eve-seg.hfst -o eve.seg.hfst
!hfst-minimise eve.seg.hfst | hfst-invert | hfst-fst2fst -O -o eve.segmenter.hfst

Пример использования сегментатора

In [35]:
word = "d'utkin"
def get_parts(word):
    good_word = word.replace("'", "\\'")
    answer = ! echo {good_word} | hfst-lookup eve.segmenter.hfst
    if answer[0].split()[2].endswith('+?'):
        parts = answer[0].split()[2].strip('+?')
    else:
        parts = answer[0].split()[2].replace('>', '-')
    return parts
get_parts(word)

"d'u-taki-{n3sg}"

Evaluation

In [10]:
import pandas as pd
df = pd.read_csv('frequency_list.csv', delimiter='\t')

In [51]:
df['auto_glosses'] = 0
df['auto_parts'] = 0
for index, row in df.iterrows():
    word = str(row['word']).replace(')', '').replace('(', '')
    df['auto_glosses'].loc[index] = get_analysis(word)
    df['auto_parts'].loc[index] = get_parts(word)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [52]:
df.to_csv('analysis.csv')

Будем дорабатывать

Основные ошибки:
- неправильный выбор из возможных (нечастотный)
- морфонология порождает не все нужные формы
- числа с суффиксами
- не разбираются слова с неизвестным корнем
 - много заимствований, которых нет в словаре

- сложность с разрешением переходов с гармонией

Осторожно, очень долго работает